## Все данные будем извлекать с двух ресурсов: [fanfics.me](fanfics.me) и [ficbook.ru](ficbook.ru)

Какие вообще данные в теории понадобятся?
- название
- ссылка
- описание фанфика
- персонажи
- пейринги
- фандомы
- возрастной рейтинг
- теги отношений (слеш/джен/гет/фем)
- тег размера (мини/миди/макси/драббл)
- размер
- статус
- события (можно получить только для фанфиков с [fanfics.me](fanfics.me))
- жанры
- предупреждения

## Что вообще происходит?

Сейчас мы будем доставать все вышеописанные данные с сайта. Для этого нужно открыть страницу со списком фанфиков и достать оттуда все названия, описания, персонажей и так далее. Чтобы произвести эти действия, нужно использовать специализированные библиотеки.

In [1]:
import requests # библиотека для доступа к исходному коду страниц
from bs4 import BeautifulSoup # библиотека для «вытаскивания» нужных нам данных из исходного кода страницы

Каждая страница с фанфикам по Гарри Поттеру на [fanfics.me](fanfics.me) выглядит следующим образом: 
>https://fanfics.me/find?fandom1=2&page=1

где последнее число отвечает за номер страницы. Всего таких страниц с фанфиками 1031 на момент написания этого кода. Давайте вытащим исходный код каждой страницы в цикле.

In [2]:
URLs = ['https://fanfics.me/find?fandom1=2&page=' + str(page) for page in range(1, 3)]
page_sources = [requests.get(url) for url in URLs]

Исходный код надо превратить в «суп», с которым будем удобно работать для поиска информации на странице. «Суп» будем искать для каждой страницы, исходный код которой мы получили. Тем самым получим список с «супами».

In [3]:
soups = [BeautifulSoup(page_source.content, 'html.parser') for page_source in page_sources]

Вся информация о фанфике содержится в контейнере 
```html
<table class='FicTbl'>
```
Поэтому для каждого «супа» найдём все разделы с информацией о фанфиках, тем самым мы получим «суп» для каждого фанфика.

In [4]:
fanfic_soups = []
for soup in soups:
    fanfic_soups += soup.find_all('table', class_='FicTbl')

## Названия
Если внимательно посмотреть на код страницы, то становится ясно, что интересующее нас название лежит в контейнере 
```html
<div class='FicTable_Title'>
``` 
так что ищем этот контейнер (он всего один для каждого «супа», поэтому используем `find` вместо `find_all`)

In [5]:
title_meta = [soup.find('div', class_='FicTable_Title') for soup in fanfic_soups]
title_meta[0]

<div class="FicTable_Title">
<h4><a href="/fic147220">Мимезис</a> <span class="small green">(гет)</span></h4>
<div class="FicTable_Stat"><span class="Views" style="margin-right:20px;" title="Количество просмотров 4 249">4.2k</span>
<span class="ReadersCount" style="margin-right:20px;" title="Количество читателей 117">117</span><span class="Comments" style="margin-right:20px;" title="Количество комментариев 138">138</span><span class="RecommendsCount" style="margin-right:20px;" title="Количество рекомендаций">1</span><span class="DateUpdate" style="margin-right:20px;" title="Опубликовано 14.06.2020, изменено 11.02.2021">11.02.2021</span></div>
</div>

Проблема в том, что контейнер содержит не только название, но и ещё массу лишней информации, поэтому находим уникальный для названия контейнер -- это контейнер `<a>`, он применяется только к названию.

In [6]:
titles = [soup.find('a').text for soup in title_meta]
titles[0]

'Мимезис'

## Ссылка

In [7]:
links = ['fanfics.me' + soup.find('a')['href'] for soup in title_meta]
links[0]

'fanfics.me/fic147220'

## Описания
Описание лежит в контейнере 
```html
<td class='FicTbl_sammary'>
``` 
так что ищем этот контейнер (он всего один для каждого «супа», поэтому используем `find` вместо `find_all`)

In [8]:
summaries = [soup.find('td', class_='FicTbl_sammary').text for soup in fanfic_soups]
summaries[0]

'Все знают, как заканчиваются сказки: Красавица влюбляется в Чудовище, которое после победы над злом превращается в Прекрасного Принца. Но в Королевстве Кривых Зеркал все наоборот, и вот привычные образы дрожат, словно миражи, превращаясь во что-то иное: стрелки часов бегут в обратную сторону, мир искажается, зло подбирается все ближе и ближе. Сказка превращается в кошмар. А что это с твоим Прекрас...>>ным Принцем? Взгляни на него, Красавица! Взгляни повнимательнее, и ты увидишь… \n\nМимезис (др. греч. - подражание, воспроизведение, подобие) - это понятие, принятое для обозначения имитации. В эстетике - искусство, подражающее действительности, копирующее реальность.'

## Персонажи

In [9]:
import re

In [10]:
characters = []
for soup in fanfic_soups:
    single = [char.text for char in soup.find_all(href=re.compile('/character.*'))]
    pairings = [pair.text.split('/') for pair in soup.find_all(href=re.compile('/paring.*'))]
    for pairing in pairings:
        single += pairing
    single = list(set(single))
    characters.append(', '.join(single))
characters[0]

'Гарри Поттер, Джинни Уизли'

## Пейринги

In [11]:
pairings = []
for soup in fanfic_soups:
    fic_pairings = [pair.text for pair in soup.find_all(href=re.compile('/paring.*'))]
    pairings.append(', '.join(fic_pairings))
pairings[0]

'Гарри Поттер/Джинни Уизли'

## Фандомы

In [12]:
fandoms = []
for soup in fanfic_soups:
    fic_fandoms = [fandom.text for fandom in soup.find_all(href=re.compile('/fandom.*'))]
    fandoms.append(', '.join(fic_fandoms))
fandoms[0]

'Гарри Поттер'

## Возрастной рейтинг

In [13]:
ratings = [soup.find('span', class_='FicTable_Rating').text for soup in fanfic_soups]
ratings[0]

'R'

## Теги отношений

In [14]:
relationships = [soup.find('div', class_='FicTable_Title').find('h4').find('span').text.strip('()') for soup in fanfic_soups]
relationships[0]

'гет'

## Теги размера + размер

In [15]:
sizes_full = [soup.find('span', class_='FicTable_Size').text.split('|') for soup in fanfic_soups]
size_tags = [size_full[0].strip() for size_full in sizes_full]
sizes = [size_full[1].strip()[:-3] for size_full in sizes_full]
(size_tags[0], sizes[0])

('Макси', '211')

## Статус

In [16]:
statuses = [soup.find('span', class_='FicTable_Status').text.replace('\xa0', ' ') for soup in fanfic_soups]
statuses[0]

'В процессе'

## События

In [17]:
events = []
for soup in fanfic_soups:
    fic_events = [event.text for event in soup.find_all(href=re.compile('/find\?keyword=.*'))]
    events.append(', '.join(fic_events))
events[0]

'ПостХогвартс, Не в Хогвартсе'

## Жанры

In [18]:
meta_unfiltered = [soup.find('td', class_='FicTbl_meta').text for soup in fanfic_soups]
meta_unfiltered[0]

'\nАвтор:\xa0Jane W.\n\nФандом:\xa0Гарри Поттер\n\nПерсонажи:\xa0Гарри Поттер/Джинни Уизли\nРейтинг:\xa0R\n\nЖанры:\xa0Ангст, Драма, Триллер\n                    \nРазмер:\xa0Макси | 211 Кб\n\nСтатус:\xa0В\xa0процессе\nПредупреждения:\xa0AU, Насилие, ООС, Сомнительное согласие\n                    События:\xa0ПостХогвартс, Не в Хогвартсе\n1\n'

In [19]:
genres_regex = re.compile('Жанры:\\xa0(.*)\\n +')

In [20]:
genres = []
for meta in meta_unfiltered:
    match = re.search(genres_regex, meta)
    if match:
        genres.append(match.group(1))
    else:
        genres.append('')
genres[0]

'Ангст, Драма, Триллер'

## Предупреждения

In [21]:
warnings_regex = re.compile('Предупреждения:\\xa0(.*)\\n +')

In [22]:
warnings = []
for meta in meta_unfiltered:
    match = re.search(warnings_regex, meta)
    if match:
        warnings.append(match.group(1))
    else:
        warnings.append('')
warnings[0]

'AU, Насилие, ООС, Сомнительное согласие'

## Делаем pandas датафрейм!

In [23]:
data = {
    'title': titles,
    'summary': summaries,
    'characters': characters,
    'pairings': pairings,
    'fandoms': fandoms,
    'rating': ratings,
    'size_tag': size_tags,
    'size': sizes,
    'statuse': statuses,
    'events': events,
    'genres': genres,
    'warnings': warnings,
    'link': links
}

In [24]:
import pandas as pd
fanfictionDF = pd.DataFrame(data, columns = data.keys())
fanfictionDF

,title,summary,characters,pairings,fandoms,rating,size_tag,size,statuse,events,genres,warnings,link
0,Мимезис,"Все знают, как заканчиваются сказки: Красавица...","Гарри Поттер, Джинни Уизли",Гарри Поттер/Джинни Уизли,Гарри Поттер,R,Макси,211,В процессе,"ПостХогвартс, Не в Хогвартсе","Ангст, Драма, Триллер","AU, Насилие, ООС, Сомнительное согласие",fanfics.me/fic147220
1,Аконит,По улицам магического Лондона бродит таинствен...,"Драко Малфой, Гарри Поттер",Гарри Поттер/Драко Малфой,Гарри Поттер,NC-17,Макси,77,В процессе,"Аврорат, Сокрытие магических способностей","Приключения, Романтика, Hurt/comfort, Драма","Нецензурная лексика, UST",fanfics.me/fic159258
2,Гарри Поттер и Хранитель магии,— Теперь ты не можешь причинить вреда Гарри По...,"Рабастан Лестрейндж, Астория Гринграсс, Родоль...",Люциус Малфой/Нарцисса Малфой/Родольфус Лестре...,Гарри Поттер,R,Макси,3657,В процессе,"Власти магического мира, Аристократия, Нестанд...","Первый раз, Драма, Приключения, Экшен","Инцест, AU, ООС, Гет, Смерть персонажа, UST",fanfics.me/fic148893
3,In saecula saeculorum,"Не стоит пробовать все, что попадется под руку...","Блейз Забини, Рон Уизли, Теодор Нотт, Гермиона...","Гарри Поттер/Северус Снейп, Невилл Лонгботтом/...",Гарри Поттер,PG-13,Макси,1154,В процессе,Анимагия,"Приключения, Романтика","AU, ООС, UST",fanfics.me/fic131140
4,Два серебряных крыла,Гарри очень торопится подарить Драко подарок н...,"Драко Малфой, Гарри Поттер",Гарри Поттер/Драко Малфой,Гарри Поттер,PG-13,Мини,8,Закончен,"Восьмой курс, Чистая романтика, Новый год и Ро...","Романтика, Флафф",,fanfics.me/fic159282
5,Запретное заклинание,"Гарри случайно узнаёт о заклинании, которое до...","Амелия Боунс, Гарри Поттер, Минерва МакГонагал...",,Гарри Поттер,R,Мини,20,Закончен,"Сокрытие магических способностей, Дамбигад, Уи...",Пародия,"Нецензурная лексика, ООС",fanfics.me/fic159107
6,Закалённые огнём,"В последний год учёбы в Хогвартсе, готовясь к ...","Рон Уизли, Гермиона Грейнджер, Джинни Уизли, С...",Гермиона Грейнджер/Северус Снейп,Гарри Поттер,NC-17,Макси,437,В процессе,"Седьмой курс, Вторая война с Волдемортом","Ангст, Драма",ООС,fanfics.me/fic154177
7,Другие берега,"А, что если Гарри по-настоящему любил только Г...","Луна Лавгуд, Рон Уизли, Сириус Блэк, Гермиона ...","Гарри Поттер/Гермиона Грейнджер, Изабелла Свон...","Гарри Поттер, Сумерки",PG-13,Макси,247,В процессе,,"Фэнтези, Кроссовер, Приключения, Романтика","AU, Гет",fanfics.me/fic71494
8,Старушка Петти,"Однажды Петунья Дурсль услышала то, что ей не ...","Гарри Поттер, Дадли Дурсль, Северус Снейп, Пет...",,Гарри Поттер,PG-13,Макси,761,В процессе,"Дурсли - волшебники, Распределение в другие фа...",Приключения,AU,fanfics.me/fic106955
9,"Пиши мне, хорошо?",Драко Малфой уезжает учиться в волшебную школу...,"Блейз Забини, Рон Уизли, Гермиона Грейнджер, П...",Гарри Поттер/Драко Малфой,Гарри Поттер,NC-17,Макси,167,В процессе,"Детство героев, Распределение в другие факультеты","Ангст, Драма, Романтика, Hurt/comfort",Нецензурная лексика,fanfics.me/fic159183


## Пришло время сделать одну большую функцию для всего этого непотребства

In [25]:
genres_regex = re.compile('Жанры:\\xa0(.*)\\n +')
warnings_regex = re.compile('Предупреждения:\\xa0(.*)\\n +')

def processPage(url):
    page_source = requests.get(url)
    soup = BeautifulSoup(page_source.content, 'html.parser')
    fanfic_soups = soup.find_all('table', class_='FicTbl')
    
    characters = []
    pairings = []
    fandoms = []
    ratings = []
    relationships = []
    size_tags = []
    sizes = []
    statuses = []
    events = []
    genres = []
    warnings = []
    titles = []
    links = []
    summaries = []
    
    for soup in fanfic_soups:
        title_meta = soup.find('div', class_='FicTable_Title')
        titles.append(soup.find('a').text)
        links.append('fanfics.me' + soup.find('a')['href'])
        
        summaries.append(soup.find('td', class_='FicTbl_sammary').text)
        
        single = [char.text for char in soup.find_all(href=re.compile('/character.*'))]
        fic_pairings = [pair.text.split('/') for pair in soup.find_all(href=re.compile('/paring.*'))]
        for fic_pairing in fic_pairings:
            single += fic_pairing
        single = list(set(single))
        characters.append(', '.join(single))
            
        fic_pairings = [pair.text for pair in soup.find_all(href=re.compile('/paring.*'))]
        pairings.append(', '.join(fic_pairings))
        
        fic_fandoms = [fandom.text for fandom in soup.find_all(href=re.compile('/fandom.*'))]
        fandoms.append(', '.join(fic_fandoms))
        
        ratings.append(soup.find('span', class_='FicTable_Rating').text)
        relationships.append(soup.find('div', class_='FicTable_Title').find('h4').find('span').text.strip('()'))
    
        size_full = soup.find('span', class_='FicTable_Size').text.split('|')
        size_tags.append(size_full[0].strip())
        sizes.append(size_full[1].strip()[:-3])
    
        statuses.append(soup.find('span', class_='FicTable_Status').text.replace('\xa0', ' '))
        
        fic_events = [event.text for event in soup.find_all(href=re.compile('/find\?keyword=.*'))]
        events.append(', '.join(fic_events))

        meta_unfiltered = soup.find('td', class_='FicTbl_meta').text
        
        genres_match = re.search(genres_regex, meta_unfiltered)
        if genres_match:
            genres.append(genres_match.group(1))
        else:
            genres.append('')
        
        warnings_match = re.search(warnings_regex, meta_unfiltered)
        if warnings_match:
            warnings.append(warnings_match.group(1))
        else:
            warnings.append('')
            
    data = {
        'title': titles,
        'summary': summaries,
        'characters': characters,
        'pairings': pairings,
        'fandoms': fandoms,
        'rating': ratings,
        'size_tag': size_tags,
        'size': sizes,
        'statuse': statuses,
        'events': events,
        'genres': genres,
        'warnings': warnings,
        'link': links
    }
    
    fanfictionDF = pd.DataFrame(data, columns = data.keys())
    return fanfictionDF

In [26]:
testDF = processPage('https://fanfics.me/find?fandom1=2&page=1')

In [27]:
testDF.to_csv('testDF.csv')
pd.read_csv('testDF.csv', index_col = 0)

,title,summary,characters,pairings,fandoms,rating,size_tag,size,statuse,events,genres,warnings,link
0,Мимезис,"Все знают, как заканчиваются сказки: Красавица...","Гарри Поттер, Джинни Уизли",Гарри Поттер/Джинни Уизли,Гарри Поттер,R,Макси,211,В процессе,"ПостХогвартс, Не в Хогвартсе","Ангст, Драма, Триллер","AU, Насилие, ООС, Сомнительное согласие",fanfics.me/fic147220
1,Аконит,По улицам магического Лондона бродит таинствен...,"Драко Малфой, Гарри Поттер",Гарри Поттер/Драко Малфой,Гарри Поттер,NC-17,Макси,77,В процессе,"Аврорат, Сокрытие магических способностей","Приключения, Романтика, Hurt/comfort, Драма","Нецензурная лексика, UST",fanfics.me/fic159258
2,Гарри Поттер и Хранитель магии,— Теперь ты не можешь причинить вреда Гарри По...,"Рабастан Лестрейндж, Астория Гринграсс, Родоль...",Люциус Малфой/Нарцисса Малфой/Родольфус Лестре...,Гарри Поттер,R,Макси,3657,В процессе,"Власти магического мира, Аристократия, Нестанд...","Первый раз, Драма, Приключения, Экшен","Инцест, AU, ООС, Гет, Смерть персонажа, UST",fanfics.me/fic148893
3,In saecula saeculorum,"Не стоит пробовать все, что попадется под руку...","Блейз Забини, Рон Уизли, Теодор Нотт, Гермиона...","Гарри Поттер/Северус Снейп, Невилл Лонгботтом/...",Гарри Поттер,PG-13,Макси,1154,В процессе,Анимагия,"Приключения, Романтика","AU, ООС, UST",fanfics.me/fic131140
4,Два серебряных крыла,Гарри очень торопится подарить Драко подарок н...,"Драко Малфой, Гарри Поттер",Гарри Поттер/Драко Малфой,Гарри Поттер,PG-13,Мини,8,Закончен,"Восьмой курс, Чистая романтика, Новый год и Ро...","Романтика, Флафф",NaN,fanfics.me/fic159282
5,Запретное заклинание,"Гарри случайно узнаёт о заклинании, которое до...","Амелия Боунс, Гарри Поттер, Минерва МакГонагал...",NaN,Гарри Поттер,R,Мини,20,Закончен,"Сокрытие магических способностей, Дамбигад, Уи...",Пародия,"Нецензурная лексика, ООС",fanfics.me/fic159107
6,Закалённые огнём,"В последний год учёбы в Хогвартсе, готовясь к ...","Рон Уизли, Гермиона Грейнджер, Джинни Уизли, С...",Гермиона Грейнджер/Северус Снейп,Гарри Поттер,NC-17,Макси,437,В процессе,"Седьмой курс, Вторая война с Волдемортом","Ангст, Драма",ООС,fanfics.me/fic154177
7,Другие берега,"А, что если Гарри по-настоящему любил только Г...","Луна Лавгуд, Рон Уизли, Сириус Блэк, Гермиона ...","Гарри Поттер/Гермиона Грейнджер, Изабелла Свон...","Гарри Поттер, Сумерки",PG-13,Макси,247,В процессе,NaN,"Фэнтези, Кроссовер, Приключения, Романтика","AU, Гет",fanfics.me/fic71494
8,Старушка Петти,"Однажды Петунья Дурсль услышала то, что ей не ...","Гарри Поттер, Дадли Дурсль, Северус Снейп, Пет...",NaN,Гарри Поттер,PG-13,Макси,761,В процессе,"Дурсли - волшебники, Распределение в другие фа...",Приключения,AU,fanfics.me/fic106955
9,"Пиши мне, хорошо?",Драко Малфой уезжает учиться в волшебную школу...,"Блейз Забини, Рон Уизли, Гермиона Грейнджер, П...",Гарри Поттер/Драко Малфой,Гарри Поттер,NC-17,Макси,167,В процессе,"Детство героев, Распределение в другие факультеты","Ангст, Драма, Романтика, Hurt/comfort",Нецензурная лексика,fanfics.me/fic159183


## Теперь надо медленно, но верно проходить по каждой странице и добавлять в датафрейм результаты парсинга

In [28]:
from datetime import datetime

In [29]:
# page_link = 'https://fanfics.me/find?fandom1=2&page='
# for page_number in range(1, 1037):
#     pageDF = processPage(page_link + str(page_number))
#     pageDF.to_csv('page_' + str(page_number) + '.csv')
#     now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
#     print(str(page_number) + ' pages processed at '+ now)

## Соединим все сохраненные файлы в один датафрейм, а потом в один файл

In [39]:
fanfictionDF = pd.read_csv('page_1.csv', index_col = 0)
for file_number in range(2, 1037):
    new = pd.read_csv('page_' + str(file_number)+ '.csv', index_col = 0)
    fanfictionDF = pd.concat([fanfictionDF, new], ignore_index = True)
    now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(str(file_number) + ' files processed at '+ now)

fanfictionDF.to_csv('fanfiction.csv')

2 files processed at 11/02/2021 00:46:33
3 files processed at 11/02/2021 00:46:33
4 files processed at 11/02/2021 00:46:33
5 files processed at 11/02/2021 00:46:33
6 files processed at 11/02/2021 00:46:33
7 files processed at 11/02/2021 00:46:33
8 files processed at 11/02/2021 00:46:33
9 files processed at 11/02/2021 00:46:33
10 files processed at 11/02/2021 00:46:33
11 files processed at 11/02/2021 00:46:33
12 files processed at 11/02/2021 00:46:33
13 files processed at 11/02/2021 00:46:33
14 files processed at 11/02/2021 00:46:33
15 files processed at 11/02/2021 00:46:33
16 files processed at 11/02/2021 00:46:33
17 files processed at 11/02/2021 00:46:33
18 files processed at 11/02/2021 00:46:33
19 files processed at 11/02/2021 00:46:33
20 files processed at 11/02/2021 00:46:33
21 files processed at 11/02/2021 00:46:33
22 files processed at 11/02/2021 00:46:33
23 files processed at 11/02/2021 00:46:33
24 files processed at 11/02/2021 00:46:33
25 files processed at 11/02/2021 00:46:33


206 files processed at 11/02/2021 00:46:36
207 files processed at 11/02/2021 00:46:36
208 files processed at 11/02/2021 00:46:36
209 files processed at 11/02/2021 00:46:36
210 files processed at 11/02/2021 00:46:36
211 files processed at 11/02/2021 00:46:36
212 files processed at 11/02/2021 00:46:36
213 files processed at 11/02/2021 00:46:36
214 files processed at 11/02/2021 00:46:36
215 files processed at 11/02/2021 00:46:36
216 files processed at 11/02/2021 00:46:36
217 files processed at 11/02/2021 00:46:36
218 files processed at 11/02/2021 00:46:36
219 files processed at 11/02/2021 00:46:36
220 files processed at 11/02/2021 00:46:36
221 files processed at 11/02/2021 00:46:36
222 files processed at 11/02/2021 00:46:36
223 files processed at 11/02/2021 00:46:36
224 files processed at 11/02/2021 00:46:36
225 files processed at 11/02/2021 00:46:36
226 files processed at 11/02/2021 00:46:36
227 files processed at 11/02/2021 00:46:36
228 files processed at 11/02/2021 00:46:36
229 files p

404 files processed at 11/02/2021 00:46:40
405 files processed at 11/02/2021 00:46:40
406 files processed at 11/02/2021 00:46:40
407 files processed at 11/02/2021 00:46:40
408 files processed at 11/02/2021 00:46:40
409 files processed at 11/02/2021 00:46:40
410 files processed at 11/02/2021 00:46:40
411 files processed at 11/02/2021 00:46:40
412 files processed at 11/02/2021 00:46:40
413 files processed at 11/02/2021 00:46:40
414 files processed at 11/02/2021 00:46:40
415 files processed at 11/02/2021 00:46:40
416 files processed at 11/02/2021 00:46:40
417 files processed at 11/02/2021 00:46:40
418 files processed at 11/02/2021 00:46:40
419 files processed at 11/02/2021 00:46:40
420 files processed at 11/02/2021 00:46:40
421 files processed at 11/02/2021 00:46:40
422 files processed at 11/02/2021 00:46:40
423 files processed at 11/02/2021 00:46:40
424 files processed at 11/02/2021 00:46:40
425 files processed at 11/02/2021 00:46:40
426 files processed at 11/02/2021 00:46:40
427 files p

596 files processed at 11/02/2021 00:46:45
597 files processed at 11/02/2021 00:46:45
598 files processed at 11/02/2021 00:46:45
599 files processed at 11/02/2021 00:46:45
600 files processed at 11/02/2021 00:46:45
601 files processed at 11/02/2021 00:46:45
602 files processed at 11/02/2021 00:46:45
603 files processed at 11/02/2021 00:46:45
604 files processed at 11/02/2021 00:46:45
605 files processed at 11/02/2021 00:46:45
606 files processed at 11/02/2021 00:46:45
607 files processed at 11/02/2021 00:46:45
608 files processed at 11/02/2021 00:46:45
609 files processed at 11/02/2021 00:46:45
610 files processed at 11/02/2021 00:46:46
611 files processed at 11/02/2021 00:46:46
612 files processed at 11/02/2021 00:46:46
613 files processed at 11/02/2021 00:46:46
614 files processed at 11/02/2021 00:46:46
615 files processed at 11/02/2021 00:46:46
616 files processed at 11/02/2021 00:46:46
617 files processed at 11/02/2021 00:46:46
618 files processed at 11/02/2021 00:46:46
619 files p

788 files processed at 11/02/2021 00:46:52
789 files processed at 11/02/2021 00:46:52
790 files processed at 11/02/2021 00:46:52
791 files processed at 11/02/2021 00:46:52
792 files processed at 11/02/2021 00:46:52
793 files processed at 11/02/2021 00:46:52
794 files processed at 11/02/2021 00:46:52
795 files processed at 11/02/2021 00:46:52
796 files processed at 11/02/2021 00:46:52
797 files processed at 11/02/2021 00:46:52
798 files processed at 11/02/2021 00:46:52
799 files processed at 11/02/2021 00:46:52
800 files processed at 11/02/2021 00:46:52
801 files processed at 11/02/2021 00:46:52
802 files processed at 11/02/2021 00:46:52
803 files processed at 11/02/2021 00:46:52
804 files processed at 11/02/2021 00:46:53
805 files processed at 11/02/2021 00:46:53
806 files processed at 11/02/2021 00:46:53
807 files processed at 11/02/2021 00:46:53
808 files processed at 11/02/2021 00:46:53
809 files processed at 11/02/2021 00:46:53
810 files processed at 11/02/2021 00:46:53
811 files p

981 files processed at 11/02/2021 00:47:01
982 files processed at 11/02/2021 00:47:01
983 files processed at 11/02/2021 00:47:01
984 files processed at 11/02/2021 00:47:01
985 files processed at 11/02/2021 00:47:01
986 files processed at 11/02/2021 00:47:01
987 files processed at 11/02/2021 00:47:01
988 files processed at 11/02/2021 00:47:01
989 files processed at 11/02/2021 00:47:01
990 files processed at 11/02/2021 00:47:01
991 files processed at 11/02/2021 00:47:01
992 files processed at 11/02/2021 00:47:01
993 files processed at 11/02/2021 00:47:01
994 files processed at 11/02/2021 00:47:01
995 files processed at 11/02/2021 00:47:01
996 files processed at 11/02/2021 00:47:01
997 files processed at 11/02/2021 00:47:01
998 files processed at 11/02/2021 00:47:01
999 files processed at 11/02/2021 00:47:01
1000 files processed at 11/02/2021 00:47:01
1001 files processed at 11/02/2021 00:47:01
1002 files processed at 11/02/2021 00:47:01
1003 files processed at 11/02/2021 00:47:02
1004 fi

In [40]:
fanfictionDF

,title,summary,characters,pairings,fandoms,rating,size_tag,size,statuse,events,genres,warnings,link
0,Digging for the Bones,После скандала с тётушкой Мардж Гарри сбежал и...,"Гермиона Грейнджер, Сириус Блэк, Северус Снейп...",NaN,Гарри Поттер,PG-13,Макси,580,В процессе,"Третий курс, Снейп - отец Гарри","Ангст, Даркфик, Hurt/comfort","AU, ООС, Насилие, Смерть персонажа",fanfics.me/fic155060
1,Цели и средства,"Симус Финниган безнадежно, отчаянно влюблен в ...","Симус Финниган, Джинни Уизли, Дин Томас",Джинни Уизли/Симус Финниган,Гарри Поттер,PG-13,Мини,26,Закончен,ПостХогвартс,"Драма, Романтика","Нецензурная лексика, ООС",fanfics.me/fic159345
2,Моя отважная лисичка,"Тёмный Лорд повержен, но Пожиратели не дремлют...","Гермиона Грейнджер, Сириус Блэк, Элис Каллен, ...","Джейкоб Блэк/Гермиона Грейнджер, Ли Клируотер ...","Сумерки, Гарри Поттер",PG-13,Макси,195,В процессе,"Оборотни, Вампиры, Рассвет, Путешествие во вре...","Ангст, AU","Гет, AU, ООС, От первого лица (POV), Смерть пе...",fanfics.me/fic159145
3,Take me home,Гарри возвращается в Хогвартс после войны в на...,"Гарри Поттер, Северус Снейп",NaN,Гарри Поттер,NC-17,Миди,78,В процессе,"Волдеморт побежден, Восьмой курс, Независимый ...","Первый раз, Романтика","AU, Гет, Нецензурная лексика",fanfics.me/fic156508
4,Великолепное лето,Гарри проводит лето со Снейпом.[2 часть цикла ...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,NC-17,Макси,83,В процессе,Летом,"Драма, Романтика","AU, ООС",fanfics.me/fic159023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20703,Наблюдая Звёзды,"Отношения Гарри и Снейпа становятся лучше, но ...","Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,NaN,NC-17,Макси,18,Заморожен,NaN,"Драма, Романтика, Первый раз",NaN,fanfics.me/fic37
20704,Time and time again,Гарри завтра женится. А сегодня ночью Драко на...,"Джинни Уизли, Драко Малфой, Гарри Поттер","Гарри Поттер/Драко Малфой, Гарри Поттер/Джинни...",NaN,R,Мини,12,Закончен,NaN,"Ангст, Романтика",NaN,fanfics.me/fic38
20705,Больше чем друзья,Дружба между юношей и девушкой рано или поздно...,"Гермиона Грейнджер, Гарри Поттер",Гарри Поттер/Гермиона Грейнджер,NaN,NC-17,Мини,16,Закончен,NaN,Романтика,NaN,fanfics.me/fic39
20706,"Люби меня, люблю тебя","Реализация замыслов хороша, когда она случайна.","Гермиона Грейнджер, Гарри Поттер",Гарри Поттер/Гермиона Грейнджер,NaN,NC-17,Мини,19,Закончен,NaN,"Романтика, Юмор",NaN,fanfics.me/fic40


In [42]:
fanfictionDF = pd.read_csv('fanfiction.csv', index_col = 0)
fanfictionDF

,title,summary,characters,pairings,fandoms,rating,size_tag,size,statuse,events,genres,warnings,link
0,Digging for the Bones,После скандала с тётушкой Мардж Гарри сбежал и...,"Гермиона Грейнджер, Сириус Блэк, Северус Снейп...",NaN,Гарри Поттер,PG-13,Макси,580,В процессе,"Третий курс, Снейп - отец Гарри","Ангст, Даркфик, Hurt/comfort","AU, ООС, Насилие, Смерть персонажа",fanfics.me/fic155060
1,Цели и средства,"Симус Финниган безнадежно, отчаянно влюблен в ...","Симус Финниган, Джинни Уизли, Дин Томас",Джинни Уизли/Симус Финниган,Гарри Поттер,PG-13,Мини,26,Закончен,ПостХогвартс,"Драма, Романтика","Нецензурная лексика, ООС",fanfics.me/fic159345
2,Моя отважная лисичка,"Тёмный Лорд повержен, но Пожиратели не дремлют...","Гермиона Грейнджер, Сириус Блэк, Элис Каллен, ...","Джейкоб Блэк/Гермиона Грейнджер, Ли Клируотер ...","Сумерки, Гарри Поттер",PG-13,Макси,195,В процессе,"Оборотни, Вампиры, Рассвет, Путешествие во вре...","Ангст, AU","Гет, AU, ООС, От первого лица (POV), Смерть пе...",fanfics.me/fic159145
3,Take me home,Гарри возвращается в Хогвартс после войны в на...,"Гарри Поттер, Северус Снейп",NaN,Гарри Поттер,NC-17,Миди,78,В процессе,"Волдеморт побежден, Восьмой курс, Независимый ...","Первый раз, Романтика","AU, Гет, Нецензурная лексика",fanfics.me/fic156508
4,Великолепное лето,Гарри проводит лето со Снейпом.[2 часть цикла ...,"Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,Гарри Поттер,NC-17,Макси,83,В процессе,Летом,"Драма, Романтика","AU, ООС",fanfics.me/fic159023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20703,Наблюдая Звёзды,"Отношения Гарри и Снейпа становятся лучше, но ...","Гарри Поттер, Северус Снейп",Гарри Поттер/Северус Снейп,NaN,NC-17,Макси,18,Заморожен,NaN,"Драма, Романтика, Первый раз",NaN,fanfics.me/fic37
20704,Time and time again,Гарри завтра женится. А сегодня ночью Драко на...,"Джинни Уизли, Драко Малфой, Гарри Поттер","Гарри Поттер/Драко Малфой, Гарри Поттер/Джинни...",NaN,R,Мини,12,Закончен,NaN,"Ангст, Романтика",NaN,fanfics.me/fic38
20705,Больше чем друзья,Дружба между юношей и девушкой рано или поздно...,"Гермиона Грейнджер, Гарри Поттер",Гарри Поттер/Гермиона Грейнджер,NaN,NC-17,Мини,16,Закончен,NaN,Романтика,NaN,fanfics.me/fic39
20706,"Люби меня, люблю тебя","Реализация замыслов хороша, когда она случайна.","Гермиона Грейнджер, Гарри Поттер",Гарри Поттер/Гермиона Грейнджер,NaN,NC-17,Мини,19,Закончен,NaN,"Романтика, Юмор",NaN,fanfics.me/fic40
